# Using MLflow REST with Azure ML

The MLflow REST API allows you to create, list, and get experiments and runs, and log parameters, metrics, and artifacts. The API is hosted under the `/api/2.0/mlflow` route on for Azure ML MLflow tracking URL.

In [ ]:
import requests

## Getting an authorization token

As with any Azure service, you need an authentication token to access the MLflow REST API. The following example shows how to get an authentication token using a service principal. You will need the tenant ID, client ID and client secret associated with the service principal. To create a service principal you can follow the steps at [Use the portal to create an Azure AD application and service principal that can access resources](https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal).

In [ ]:
tenant_id = "<TENANT_ID>"
client_id = "<CLIENT_ID>"
client_secret = "<CLIENT_SECRET>"
grant_type = "client_credentials"
resource_scope = "https://management.azure.com"

The request URL is as follows:

In [ ]:
mso_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token?"

The service principal information is included inside the body:

In [ ]:
mso_body = f"grant_type={grant_type}&client_id={client_id}&client_secret={client_secret}&resource={resource_scope}"

Making the request and getting the token back:

In [ ]:
auth_response = requests.post(url=mso_url, data=mso_body)

In [ ]:
token_data = auth_response.json()

> Token is returned in the field `access_token`.

## Querying the MLflow API

You can get the MLflow API from the Azure portal, or using the MLflow SDK as follows:

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

You can use the workspace object to get the tracking URI:

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [ ]:
azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri

> You can get this URL following this steps: Navigate to Azure ML Studio and select the workspace you are working on > Click on the name of the workspace at the upper right corner of the page. > Click “View all properties in Azure Portal” on the pane popup. > Copy the MLflow tracking URI value from the properties section. Then replace the protocol from `azureml` to `https`.

Once you have the tracking URI, the MLflow service URI is exactly the same, but with a different protocol:

In [ ]:
azure_ml_mlflow_base = azureml_tracking_uri.replace("azureml://", "https://")

After the base path, the service is available in the following route:

In [ ]:
mlflow_api_path = "api/2.0/mlflow"

Construct the request headers including the authentication token you just got:

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"{token_data['token_type']} {token_data['access_token']}",
}

Construct the request URL as indicated at https://mlflow.org/docs/latest/rest-api.html. For example, the following URL list all the experiments in the workspace:

In [ ]:
response = requests.get(
    f"{azure_ml_mlflow_base}/{mlflow_api_path}/experiments/list", headers=headers
)

In the following example, we are retrieving the experiments and parsing the results into a Pandas DataFrame for visualization:

In [ ]:
data = response.json()

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(data["experiments"])